In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob2
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import math
from skimage import exposure

# SVM

In [2]:
class SVM():
  def __init__(self,C=1.0):
    # C error terms
    self.C = C
    self.w = 0
    self.b = 0

  # Hinge Loss Function / Calculation
  def hingeloss(self, w, b, x, y):
    # Regularizer term
    reg = 0.5 * (w * w)

    for i in range(x.shape[0]):
      # Optimization term
      opt_term = y[i] * ((np.dot(w, x[i])) + b)

      # calculating loss
      loss = reg + self.C * max(0, 1-opt_term)
    return loss[0][0]

  def fit(self, X, Y, batch_size=100, learning_rate=0.001, epochs=1000):
    # The number of features in X
    number_of_features = X.shape[1]

    # The number of Samples in X
    number_of_samples = X.shape[0]

    c = self.C

    # Creating ids from 0 to number_of_samples - 1
    ids = np.arange(number_of_samples)

    # Shuffling the samples randomly
    np.random.shuffle(ids)

    # creating an array of zeros
    w = np.zeros((1, number_of_features))
    b = 0
    losses = []

    # Gradient Descent logic
    for i in range(epochs):
      # Calculating the Hinge Loss
      l = self.hingeloss(w, b, X, Y)

      # Appending all losses 
      losses.append(l)
      
      # Starting from 0 to the number of samples with batch_size as interval
      for batch_initial in range(0, number_of_samples, batch_size):
        gradw = 0
        gradb = 0

        for j in range(batch_initial, batch_initial + batch_size):
          if j < number_of_samples:
            x = ids[j]
            ti = Y[x] * (np.dot(w, X[x].T) + b)

            if ti > 1:
              gradw += 0
              gradb += 0
            else:
              # Calculating the gradients
              #w.r.t w 
              gradw += c * Y[x] * X[x]
              # w.r.t b
              gradb += c * Y[x]

        # Updating weights and bias
        w = w - learning_rate * w + learning_rate * gradw
        b = b + learning_rate * gradb

    self.w = w
    self.b = b

    return self.w, self.b, losses

  def predict(self, X):
    prediction = np.dot(X, self.w[0]) + self.b # w.x + b
    return np.sign(prediction)

# HOG

In [3]:
class HOG:
    def __init__(self, blockSize, cellSize, nbins, sbins):
        self.blockSize     = blockSize
        self.cellSize      = cellSize
        self.nbins         = nbins
        self.sbins         = sbins
    def __gray(self):
        picture = self.picture_array
        self.gray = 0.299*picture[:,:,0]+0.587*picture[:,:,1]+0.114*picture[:,:,2]
    def __calc_gradient(self):
        input_   = self.gray
        output_x = np.zeros((self.height,self.width))
        output_y = np.zeros((self.height,self.width))

        for r in range(self.height):
            for c in range(self.width):

                for i in range(-1,2):
                    pixel_r = r + i
                    pixel_r = min(max(0, pixel_r), self.height - 1)
                    output_y[r,c] += input_[pixel_r,c] * i

                    pixel_c = c + i
                    pixel_c = min(max(0, pixel_c), self.width - 1)
                    output_x[r,c] += input_[r,pixel_c] * i
        return output_x, output_y
    def __calc_direc_mag(self):
        self.__gray()
        gradient_x, gradient_y = self.__calc_gradient()
        self.magnitude = np.sqrt(np.square(gradient_x)+np.square(gradient_y))
        self.direction = np.mod(np.add(360, np.rad2deg(np.arctan2(np.array(gradient_y), np.array(gradient_x)))), 360)
    def __cell_hist(self,idx, idy):
        output = np.zeros(9) # output = [0,0,0,0,0,0,0,0,0]

        # duyệt qua kích thước cell theo chiều cao
        for r in range(self.cellSize[0]):
          # duyệt qua kích thước cell theo chiều rộng
            for c in range(self.cellSize[1]):
              # cột và dòng hiện tại trong ảnh
                cur_r = idy*self.cellSize[0] + r
                cur_c = idx*self.cellSize[1] + c
                # kiểm tra
                if cur_r>=self.height or cur_c >= self.width:
                    break

                # chia lấy phần nguyên và phần dư
                quotient = int(self.direction[cur_r][cur_c]//self.sbins)
                remainder = self.direction[cur_r][cur_c] % self.sbins

                if remainder==0:
                    output[quotient] += self.magnitude[cur_r][cur_c]
                else:
                    first_bin = quotient

                    second_bin = first_bin+1

                    output[first_bin] += self.magnitude[cur_r][cur_c]*\
                        ((second_bin*self.sbins - self.direction[cur_r][cur_c])/ \
                                         (second_bin*self.sbins - first_bin*self.sbins))

                    second_bin_idx = second_bin
                    if second_bin > 8:
                        second_bin_idx = 0
                    output[second_bin_idx] += self.magnitude[cur_r][cur_c]*\
                        ((self.direction[cur_r][cur_c] - first_bin*self.sbins)/(second_bin*self.sbins - first_bin*self.sbins))
        return output

    def __all_hist(self):
        hist = []
        for y in range(0,self.n_cell[0]):
            row = []
            for x in range(0,self.n_cell[1]):
                output = self.__cell_hist(x,y)
                row.append(output)
            hist.append(row)
        self.hist = np.array(hist)

        
    def compute_HOG(self, picture):
        self.picture_array = picture
        self.height, self.width, self.channel = self.picture_array.shape
        self.n_cell  = (self.height//self.cellSize[0], self.width//self.cellSize[1])
        self.n_block = (self.n_cell[0] - self.blockSize[0] + 1, self.n_cell[1] - self.blockSize[1] + 1)
        
        self.__calc_direc_mag()
        self.__all_hist()
        norm_array_size = self.n_block[0] * self.n_block[1] * self.blockSize[0] * self.blockSize[1] * self.nbins
        l2 = np.empty(self.n_block)
        for i in range(self.n_block[0]):
            for j in range(self.n_block[1]):
                l2[i][j] = math.sqrt(np.sum(np.square(self.hist[i:i+2, j:j+2])))
        norm_block = np.zeros((self.n_block[0], self.n_block[1], self.blockSize[0], self.blockSize[1], self.nbins))
        for y in range(self.n_block[0]):
            for x in range(self.n_block[1]):
                out = self.hist[y: y + self.blockSize[0], x: x + self.blockSize[1]] / (l2[y][x] + 1)
                norm_block[y][x] = out
        self.HOG        = norm_block.flatten()
        self.norm_block = norm_block
        return self.HOG

# Đọc data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cell_size = (8,8)

In [6]:
list_cat = glob2.glob('drive/MyDrive/ltssud/data/Cat/**')
images = []
labels = []

for i in range(5):
    img = Image.open(list_cat[i])
    img = img.resize((400,400), Image.LANCZOS)
    images.append(np.array(img))
    labels.append(1)

In [7]:
list_dog = glob2.glob('drive/MyDrive/ltssud/data/Dog/**')

for i in range(5):
    img = Image.open(list_dog[i])
    img = img.resize((400,400), Image.LANCZOS)
    images.append(np.array(img))
    labels.append(-1)

In [8]:
X = np.array(images)
y = np.array(labels)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
def big_fit(X_train, y_train, hog, svm):
  x_train = []
  t = 0
  for x in X_train:
    print(t)
    x_train.append(hog.compute_HOG(x))
    t+=1
  x_train = np.array(x_train)
  svm.fit(x_train, y_train)

def big_predict(X_test, y_test, hog, svm):
  x_test = []
  t = 0
  for x in X_test:
    print(t)
    x_test.append(hog.compute_HOG(x))
    t+=1
  x_test = np.array(x_test)
  return svm.predict(x_test)

In [11]:
# hog = HOG((2,2), (8,8), 9, 40)
# svm = SVM()

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

In [13]:
# svm = SVC(kernel='linear', C=1.0) 

In [1]:
import tensorflow as tf
from scipy import ndimage
from shutil import copyfile
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [ ]:
inputs = tf.keras.layers.Input(shape=(400,400,3))
x =  tf.keras.layers.Conv2D(32, (3,3), activation='relu')(inputs)
x = tf.keras.layers.Conv2D(64, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2,2)(x)

x = tf.keras.layers.Conv2D(64, (3,3), activation='relu')(x)
x = tf.keras.layers.Conv2D(128, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2,2)(x)

x = tf.keras.layers.Conv2D(128, (3,3), activation='relu')(x)
x = tf.keras.layers.Conv2D(256, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2,2)(x)


x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x) 
x = tf.keras.layers.Dense(2, activation='softmax')(x) 

model = Model(inputs=inputs, outputs=x)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# X_train = tf.keras.layers.Rescaling(1./255)

In [ ]:
y_t = np.array([0 if x==-1 else 1 for x in y_train])
y_te = np.array([0 if x==-1 else 1 for x in y_test])

In [ ]:
history = model.fit(X_train, y_t, batch_size = 10,
	epochs=10, verbose=1)